In [1]:
import quippy

At the moment, the pythonic wrapper around the descriptors fortran module still needs to be imported explicitly (hopefully this will be an invisible wrapper in the future, so that you can just do `from quippy import descriptors`).

In [2]:
from quippy import descriptors

Create a configuration:

In [3]:
at = quippy.diamond(2)
at.Z[:] = 6
at.positions

array([[ 0. ,  0. ,  0. ],
       [ 0.5,  0.5,  0.5],
       [ 1. ,  1. ,  0. ],
       [ 1.5,  1.5,  0.5],
       [ 1. ,  0. ,  1. ],
       [ 1.5,  0.5,  1.5],
       [ 0. ,  1. ,  1. ],
       [ 0.5,  1.5,  1.5]])

All descriptors are instantiated by a call to `Descriptor()`, which takes the descriptor initialisation string as its only argument. For a list of available descriptors and their parameters, see []

Here we use a simple pair distance between carbon atoms, with a cutoff of 1.5 Angstrom.

In [4]:
desc = descriptors.Descriptor("distance_2b Z1=6 Z2=6 cutoff=2")

The descriptor dimension is the length of the descriptor vector. For the scalar distance this is 1.

This descriptor is very simple: it is scalar (dimension 1), and hence only has a single permutation.

In [5]:
desc.n_dim # number of dimensions

1

In [6]:
desc.n_perm # number of permutations

1

In [7]:
desc.permutations() # array of permutation arrays

array([[1]], dtype=int32)

Many descriptors rely on the neighbour connectivity, so we need to call `calc_connect`, after setting the Atoms cutoff with a skin of 1 A:

In [8]:
at.set_cutoff(desc.cutoff()+1.0)
at.calc_connect()

We can now calculate how many instances of this descriptor are found in an `Atoms` (or `ASEAtoms`) object:

In [9]:
desc.count(at)

224

This also works transparently for iterables (such as lists), returning a list of the counts.

We can also calculate the actual descriptor values -- in this case, the list of pairwise distances:

In [10]:
desc.calc(at)

{'cutoff': array([ 0.77237468,  0.77237468,  0.77237468,  0.77237468,  0.77237468,
         0.77237468,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  0.77237468,  1.        ,
         0.77237468,  1.        ,  0.77237468,  1.        ,  0.77237468,
         1.        ,  1.        ,  0.77237468,  1.        ,  0.77237468,
         1.        ,  1.        ,  1.        ,  0.77237468,  0.77237468,
         0.77237468,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  0.77237468,  1.        ,  0.77237468,  1.        ,
         0.77237468,  1.        ,  0.77237468,  1.        ,  1.        ,
         1.        ,  0.77237468,  1.        ,  0.77237468,  1.        ,
         1.        ,  1.        ,  1.        ,  0.77237468,  0.77237468,
         0.77237468,  0.77237468,  0.77237468,  1.        ,  0.77237468,
         1.        ,  1.        ,  1.        ,  1.        ,  0.77237468,
         0.77237468,  0.77237468,  1.    

[to be moved into wrapper]
Calculate size of descriptor data:

In [11]:
n_desc, n_cross = desc.descriptor_sizes(at)
print("n_desc=%d n_cross=%d" % (n_desc,n_cross))

n_desc=224 n_cross=448


`n_desc` is number of descriptors, `n_cross` is number of gradients

Gradients are returned in a `DescriptorCalcResult` object, which has the following elements:
- `descriptor` = the descriptor values (equivalent to `desc.calc(at)`)
- `grad` = the gradients
- `index` = the indices (`i_desc`, `i_atom`, `i_coord`)

In [12]:
res = desc.calc(at, grad=True)
list(res)

['descriptor', 'cutoff_grad', 'grad', 'index', 'cutoff']

In [13]:
res.descriptor[:,:10]

array([[ 1.6583124 ],
       [ 1.6583124 ],
       [ 1.6583124 ],
       [ 1.6583124 ],
       [ 1.6583124 ],
       [ 1.6583124 ],
       [ 0.8660254 ],
       [ 1.41421356],
       [ 0.8660254 ],
       [ 1.41421356],
       [ 0.8660254 ],
       [ 1.41421356],
       [ 0.8660254 ],
       [ 1.6583124 ],
       [ 1.41421356],
       [ 1.6583124 ],
       [ 1.41421356],
       [ 1.6583124 ],
       [ 1.41421356],
       [ 1.6583124 ],
       [ 1.41421356],
       [ 1.41421356],
       [ 1.6583124 ],
       [ 1.41421356],
       [ 1.6583124 ],
       [ 1.41421356],
       [ 1.41421356],
       [ 1.41421356],
       [ 1.6583124 ],
       [ 1.6583124 ],
       [ 1.6583124 ],
       [ 0.8660254 ],
       [ 1.41421356],
       [ 1.41421356],
       [ 1.41421356],
       [ 1.41421356],
       [ 1.6583124 ],
       [ 1.41421356],
       [ 1.6583124 ],
       [ 1.41421356],
       [ 1.6583124 ],
       [ 1.41421356],
       [ 1.6583124 ],
       [ 1.41421356],
       [ 0.8660254 ],
       [ 1

In [14]:
res.grad[:,:10]

array([[[ 0.30151134],
        [ 0.30151134],
        [ 0.90453403]],

       [[-0.30151134],
        [-0.30151134],
        [-0.90453403]],

       [[-0.30151134],
        [-0.30151134],
        [ 0.90453403]],

       ..., 
       [[-0.30151134],
        [-0.90453403],
        [ 0.30151134]],

       [[ 0.30151134],
        [-0.30151134],
        [ 0.90453403]],

       [[-0.30151134],
        [ 0.30151134],
        [-0.90453403]]])

In [15]:
res.index[:,:10]

array([[  1,   1],
       [  1,   4],
       [  2,   1],
       [  2,   2],
       [  3,   1],
       [  3,   6],
       [  4,   1],
       [  4,   2],
       [  5,   1],
       [  5,   8],
       [  6,   1],
       [  6,   2],
       [  7,   1],
       [  7,   8],
       [  8,   1],
       [  8,   7],
       [  9,   1],
       [  9,   6],
       [ 10,   1],
       [ 10,   5],
       [ 11,   1],
       [ 11,   4],
       [ 12,   1],
       [ 12,   3],
       [ 13,   1],
       [ 13,   2],
       [ 14,   1],
       [ 14,   6],
       [ 15,   1],
       [ 15,   5],
       [ 16,   1],
       [ 16,   4],
       [ 17,   1],
       [ 17,   3],
       [ 18,   1],
       [ 18,   8],
       [ 19,   1],
       [ 19,   7],
       [ 20,   1],
       [ 20,   4],
       [ 21,   1],
       [ 21,   3],
       [ 22,   1],
       [ 22,   3],
       [ 23,   1],
       [ 23,   8],
       [ 24,   1],
       [ 24,   7],
       [ 25,   1],
       [ 25,   6],
       [ 26,   1],
       [ 26,   5],
       [ 27,

Now for a more interesting descriptor: a three-site benzene monomer

In [16]:
desc_monomer = descriptors.Descriptor('general_monomer signature={6 6 6} atom_ordercheck=F')

Three intramolecular distances = dimensionality 3:

In [17]:
desc_monomer.n_dim

3

And a lot more permutations now:

In [18]:
desc_monomer.n_perm

6

In [19]:
desc_monomer.permutations()

array([[1, 2, 3],
       [2, 1, 3],
       [1, 3, 2],
       [3, 1, 2],
       [2, 3, 1],
       [3, 2, 1]], dtype=int32)

Load some test configurations

In [20]:
benzat = quippy.AtomsList('benzene_frames.xyz')

In [21]:
benzat.pos.shape

(40, 3, 648)

In [22]:
for i in range(len(benzat)):
    benzat[i].set_cutoff(desc_monomer.cutoff()+0.5)
    benzat[i].calc_connect()

Calling `calc` or any of the other methods on an `AtomsList` returns a list of results. Here we look at the results for the first configuration:

In [23]:
res = desc_monomer.calc(benzat, grad=True)[0]

In [24]:
res.grad.shape

(648, 3, 3)

In [25]:
import numpy as np

In [26]:
np.array(res.index)[2]

array([1, 3], dtype=int32)

In [27]:
res.index[1] == res.index[:,1]

/usr/lib64/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


False

In [28]:
res.index[:2] == res.index[:2,:]

array([[ True,  True],
       [ True,  True]], dtype=bool)

In [29]:
res.index[:2] == res.indeax[:,:2]

KeyError: 'indeax'

In [ ]:
res.index[:10] == res.index[:,:10]